# CAFQA

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize, claptonize_opt, claptonize_opt2
from clapton.ansatzes import circular_ansatz
import numpy as np

d:\research-impl\clapton-cafqa-opt\clapton\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
np.random.seed(42)  # You can replace 42 with any integer seed
coeffs = np.random.random(len(paulis))
# coeffs = np.ones((len(paulis)))
print(coeffs)

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]


In [3]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [4]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [5]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize_opt2(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

[I 2025-07-17 23:05:28,323] A new study created in memory with name: no-name-fa757330-9bb9-4a3a-9f68-e252fe6285a5


[I 2025-07-17 23:05:28,335] Trial 0 finished with value: 0.0 and parameters: {'x0': 1, 'x1': 3, 'x2': 2, 'x3': 2, 'x4': 0, 'x5': 0, 'x6': 0, 'x7': 3, 'x8': 2, 'x9': 2, 'x10': 0, 'x11': 3}. Best is trial 0 with value: 0.0.
[I 2025-07-17 23:05:28,345] Trial 1 finished with value: 0.0 and parameters: {'x0': 3, 'x1': 0, 'x2': 0, 'x3': 0, 'x4': 1, 'x5': 2, 'x6': 1, 'x7': 1, 'x8': 2, 'x9': 0, 'x10': 1, 'x11': 1}. Best is trial 0 with value: 0.0.
[I 2025-07-17 23:05:28,358] Trial 2 finished with value: -0.39212260375335384 and parameters: {'x0': 1, 'x1': 3, 'x2': 0, 'x3': 2, 'x4': 2, 'x5': 0, 'x6': 2, 'x7': 0, 'x8': 0, 'x9': 3, 'x10': 3, 'x11': 3}. Best is trial 2 with value: -0.39212260375335384.
[I 2025-07-17 23:05:28,371] Trial 3 finished with value: 0.0 and parameters: {'x0': 1, 'x1': 0, 'x2': 2, 'x3': 1, 'x4': 0, 'x5': 1, 'x6': 0, 'x7': 3, 'x8': 1, 'x9': 2, 'x10': 1, 'x11': 2}. Best is trial 2 with value: -0.39212260375335384.
[I 2025-07-17 23:05:28,380] Trial 4 finished with value: 0.0 

[2, 1, 2, 3, 0, 0, 3, 1, 3, 0, 3, 1]


In [6]:
# the best parameters are
ks_best

[2, 1, 2, 3, 0, 0, 3, 1, 3, 0, 3, 1]

In [7]:
# with energy
energy_best

np.float64(-1.7053673109431555)